In [1]:
# from google.colab import drive
# drive.mount('/content/drive')

# Summury

In this part, we are merging all the different table in order to have one dataset. For that, we created classes for each dataset to do some preprocessing, data cleaning and feature engineering. 

# Preprocessing

In [2]:
%reload_ext autoreload
%autoreload 2
import sys
from pathlib import Path
import warnings
from datetime import datetime
from pathlib import Path

import cufflinks as cf
import pandas as pd
import pendulum
from loguru import logger
from plotly.offline import init_notebook_mode

sys.path.append(str(Path.cwd().parent))
from settings.params import PARAMS, SEED
from src.load_all_dataset import load_all_dataset
from src.utils import reduce_memory_usage
from src.datasets import *


init_notebook_mode(connected=True)
cf.go_offline()


warnings.filterwarnings('ignore')


pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", 100)

In [3]:
# Set logging format
log_fmt = "<green>{time:YYYY-MM-DD HH:mm:ss.SSS!UTC}</green> | <level>{level: <8}</level> | <cyan>{name}</cyan>:<cyan>{function}</cyan>:<cyan>{line}</cyan> - {message}"
logger.configure(handlers=[{"sink": sys.stderr, "format": log_fmt}])

# current date
CURRENT_DATE = pendulum.now(tz="UTC")

In [4]:
DIR = PARAMS['file_directory']

In [5]:
DIR = "data/home-credit-default-risk/"

In [6]:
CLEANED_DATA_DIR = PARAMS['cleaned_data_directory']

# Preprocessing of each dataset

## Bureau and Bureau_balance

In [7]:
bureau_aggregated = preprocess_bureau_balance_and_bureau(file_directory=DIR).main()

2024-08-07 17:56:34.714 | INFO     | bureau:__init__:47 - Preprocessing class initialized.
2024-08-07 17:56:34.719 | INFO     | bureau:preprocess_bureau_balance:63 - #######################################################
2024-08-07 17:56:34.724 | INFO     | bureau:preprocess_bureau_balance:64 - #          Pre-processing bureau_balance.csv          #
2024-08-07 17:56:34.728 | INFO     | bureau:preprocess_bureau_balance:65 - #######################################################
2024-08-07 17:56:34.730 | INFO     | bureau:preprocess_bureau_balance:66 - 
Loading the DataFrame, bureau_balance.csv, into memory...
2024-08-07 17:56:50.785 | INFO     | utils:reduce_memory_usage:25 - Memory usage of dataframe is 624.85 MB
2024-08-07 17:56:51.371 | INFO     | utils:reduce_memory_usage:56 - Memory usage after optimization is: 338.46 MB
2024-08-07 17:56:51.371 | INFO     | utils:reduce_memory_usage:57 - Decreased by 45.8%
2024-08-07 17:56:51.376 | INFO     | bureau:preprocess_bureau_balance:71 -

## Previous Application

In [9]:
previous_aggregated = preprocess_previous_application(file_directory=CLEANED_DATA_DIR).main()

2024-08-07 17:58:42.249 | INFO     | previous_application:__init__:43 - Preprocessing class initialized.
2024-08-07 17:58:42.250 | INFO     | previous_application:load_dataframe:53 - ########################################################
2024-08-07 17:58:42.251 | INFO     | previous_application:load_dataframe:54 - #        Pre-processing previous_application.csv        #
2024-08-07 17:58:42.252 | INFO     | previous_application:load_dataframe:55 - ########################################################
2024-08-07 17:58:42.253 | INFO     | previous_application:load_dataframe:56 - Loading the DataFrame, previous_application.csv, into memory...
2024-08-07 17:58:56.840 | INFO     | utils:reduce_memory_usage:25 - Memory usage of dataframe is 471.48 MB
2024-08-07 17:58:57.536 | INFO     | utils:reduce_memory_usage:56 - Memory usage after optimization is: 293.08 MB
2024-08-07 17:58:57.536 | INFO     | utils:reduce_memory_usage:57 - Decreased by 37.8%
2024-08-07 17:58:57.536 | INFO     | pr

## installments_payments

In [10]:
installments_aggregated = preprocess_installments_payments(file_directory=DIR).main()

2024-08-07 17:59:08.865 | INFO     | installements_payments:__init__:43 - Preprocessing class initialized.
2024-08-07 17:59:08.865 | INFO     | installements_payments:load_dataframe:53 - ##########################################################
2024-08-07 17:59:08.865 | INFO     | installements_payments:load_dataframe:54 - #        Pre-processing installments_payments.csv        #
2024-08-07 17:59:08.865 | INFO     | installements_payments:load_dataframe:55 - ##########################################################
2024-08-07 17:59:08.865 | INFO     | installements_payments:load_dataframe:56 - Loading the DataFrame, installments_payments.csv, into memory...
2024-08-07 17:59:34.701 | INFO     | utils:reduce_memory_usage:25 - Memory usage of dataframe is 830.41 MB
2024-08-07 17:59:38.844 | INFO     | utils:reduce_memory_usage:56 - Memory usage after optimization is: 311.40 MB
2024-08-07 17:59:38.850 | INFO     | utils:reduce_memory_usage:57 - Decreased by 62.5%
2024-08-07 17:59:38.856

## POS_CASH_balance

In [11]:
pos_aggregated = preprocess_POS_CASH_balance(file_directory=DIR).main()

2024-08-07 17:59:49.552 | INFO     | pos_cash:__init__:39 - Preprocessing class initialized.
2024-08-07 17:59:49.568 | INFO     | pos_cash:load_dataframe:49 - #########################################################
2024-08-07 17:59:49.568 | INFO     | pos_cash:load_dataframe:50 - #          Pre-processing POS_CASH_balance.csv          #
2024-08-07 17:59:49.568 | INFO     | pos_cash:load_dataframe:51 - #########################################################
2024-08-07 17:59:49.592 | INFO     | pos_cash:load_dataframe:52 - Loading the DataFrame, POS_CASH_balance.csv, into memory...
2024-08-07 18:00:07.474 | INFO     | utils:reduce_memory_usage:25 - Memory usage of dataframe is 610.43 MB
2024-08-07 18:00:08.638 | INFO     | utils:reduce_memory_usage:56 - Memory usage after optimization is: 238.45 MB
2024-08-07 18:00:08.638 | INFO     | utils:reduce_memory_usage:57 - Decreased by 60.9%
2024-08-07 18:00:08.638 | INFO     | pos_cash:load_dataframe:58 - Loaded POS_CASH_balance.csv
2024-08

## credit_card_balance

In [12]:
cc_aggregated = preprocess_credit_card_balance(file_directory=DIR).main()

2024-08-07 18:00:25.357 | INFO     | credit_card_balance:__init__:37 - Preprocessing class initialized.
2024-08-07 18:00:25.357 | INFO     | credit_card_balance:load_dataframe:47 - #########################################################
2024-08-07 18:00:25.357 | INFO     | credit_card_balance:load_dataframe:48 - #        Pre-processing credit_card_balance.csv         #
2024-08-07 18:00:25.357 | INFO     | credit_card_balance:load_dataframe:49 - #########################################################
2024-08-07 18:00:25.370 | INFO     | credit_card_balance:load_dataframe:50 - Loading the DataFrame, credit_card_balance.csv, into memory...
2024-08-07 18:00:44.984 | INFO     | utils:reduce_memory_usage:25 - Memory usage of dataframe is 673.88 MB
2024-08-07 18:00:47.150 | INFO     | utils:reduce_memory_usage:56 - Memory usage after optimization is: 289.33 MB
2024-08-07 18:00:47.150 | INFO     | utils:reduce_memory_usage:57 - Decreased by 57.1%
2024-08-07 18:00:47.150 | INFO     | credit

## application_train and application_test

In [14]:
application_train, application_test = preprocess_application_train_test(
    file_directory1=CLEANED_DATA_DIR, file_directory2=DIR
).main()

2024-08-07 18:02:57.776 | INFO     | application:load_dataframes:56 - #######################################################
2024-08-07 18:02:57.779 | INFO     | application:load_dataframes:57 - #        Pre-processing application_train.csv         #
2024-08-07 18:02:57.782 | INFO     | application:load_dataframes:58 - #        Pre-processing application_test.csv          #
2024-08-07 18:02:57.784 | INFO     | application:load_dataframes:59 - #######################################################
2024-08-07 18:02:57.787 | INFO     | application:load_dataframes:60 - 
Loading the DataFrames into memory...
2024-08-07 18:03:16.154 | INFO     | utils:reduce_memory_usage:25 - Memory usage of dataframe is 288.57 MB
2024-08-07 18:03:17.849 | INFO     | utils:reduce_memory_usage:56 - Memory usage after optimization is: 92.38 MB
2024-08-07 18:03:17.853 | INFO     | utils:reduce_memory_usage:57 - Decreased by 68.0%
2024-08-07 18:03:20.321 | INFO     | utils:reduce_memory_usage:25 - Memory usage

In [15]:
train_data, test_data = merge_all_tables(
    application_train,
    application_test,
    bureau_aggregated,
    previous_aggregated,
    installments_aggregated,
    pos_aggregated,
    cc_aggregated,
)

2024-08-07 18:03:36.801 | INFO     | merge_all_tables:merge_all_tables:34 - Merging application_train and application_test with aggregated tables.
2024-08-07 18:03:37.811 | INFO     | merge_all_tables:merge_all_tables:39 - Merged with bureau_aggregated.
2024-08-07 18:03:41.023 | INFO     | merge_all_tables:merge_all_tables:44 - Merged with previous_aggregated.
2024-08-07 18:03:43.339 | INFO     | merge_all_tables:merge_all_tables:49 - Merged with installments_aggregated.
2024-08-07 18:03:45.168 | INFO     | merge_all_tables:merge_all_tables:54 - Merged with pos_aggregated.
2024-08-07 18:03:50.633 | INFO     | merge_all_tables:merge_all_tables:63 - Filled missing values with 0.
2024-08-07 18:03:52.269 | INFO     | merge_all_tables:merge_all_tables:68 - Cleaned column names.
2024-08-07 18:03:53.312 | INFO     | merge_all_tables:merge_all_tables:82 - Ensured train and test data have the same columns.
2024-08-07 18:03:54.258 | INFO     | merge_all_tables:merge_all_tables:87 - Removed SK_ID

In [ ]:
train_data.to_csv(DIR + 'train_data_final.csv')
test_data.to_csv(DIR + 'test_data_final.csv')